In [ ]:
import os

!pip install keras4torch
!pip install einops

import keras4torch
from   keras4torch.callbacks  import ModelCheckpoint,LRScheduler
import torch
import torch.nn    as nn
import torch.optim as optim
import  torch.nn.functional as     F
import numpy       as np
import pandas      as pd
from copy import deepcopy
import  matplotlib.pyplot   as     plt
from    sklearn.preprocessing import StandardScaler, QuantileTransformer
from    datetime import datetime
import  gc
import STab
from STab import *
from   STab import mainmodel, LWTA, Gsoftmax
MainModel=mainmodel.MainModel

In [ ]:
##Load DAta
X_test_N=pd.DataFrame(np.load('Data/adult/N_test.npy')).astype(np.float32)
X_test_C=pd.DataFrame(np.load('Data/adult/C_test.npy'))
y_test=pd.DataFrame(np.load('Data/adult/y_test.npy')).astype(np.int32)[0]

X_train_N=pd.DataFrame(np.load('Data/adult/N_train.npy')).astype(np.float32)
X_train_C=pd.DataFrame(np.load('Data/adult/C_train.npy'))
Y_train=pd.DataFrame(np.load('Data/adult/y_train.npy')).astype(np.int32)[0]


X_valid_N=pd.DataFrame(np.load('Data/adult/N_val.npy')).astype(np.float32)
X_valid_C=pd.DataFrame(np.load('Data/adult/C_val.npy'))
y_valid=pd.DataFrame(np.load('Data/adult/y_val.npy')).astype(np.int32)[0]



In [ ]:
catmap=CatMap(X_train_C)
X_train_C=catmap(X_train_C)
X_valid_C=catmap(X_valid_C)
X_test_C=catmap(X_test_C)

In [ ]:
#Normalise In
scalerX = StandardScaler()

scalerX.fit(X_train_N)

X_train_N = scalerX.transform(X_train_N).astype(np.float32)
X_valid_N = scalerX.transform(X_valid_N).astype(np.float32)
X_test_N  = scalerX.transform(X_test_N).astype(np.float32)

In [ ]:
#combine numerical and categorical
X_train = np.concatenate([X_train_N,X_train_C.values],axis=1)
X_test  = np.concatenate([X_test_N,X_test_C.values],axis=1)
X_valid = np.concatenate([X_valid_N,X_valid_C.values],axis=1)

In [ ]:
checkpoint='saved/savefileAD'

In [ ]:
def True_ACC(p,t):
                return np.mean(np.where(np.round(p)==np.round(t),1,0))

In [ ]:
Or_model = MainModel(
    categories        = (9, 16,  7 , 15,6  ,  5,  2, 42),     # setting up all classes per categorical feature
    num_continuous    = 6,
    dim               = 16  ,
    dim_out           = 2,
    depth             = 3,
    heads             = 8,
    attn_dropout      = 0.125,
    ff_dropout        = 0.125,
    U                 = 2,
    cases             = 16,
)

#wrappers allows size N prediction sample averaging
no_model = Num_Cat(Or_model,num_number=6,classes=2,Sample_size=16)
model    = keras4torch.Model(no_model,).build([14])

#Warm Up Train
optimizer=torch.optim.AdamW(model.parameters(),lr=0.0001,weight_decay=0.0001,)
sch=torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.1, total_iters=10,  verbose=False)
model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy],)
callbacks=[ModelCheckpoint(checkpoint,monitor='val_acc',mode='max'),LRScheduler(sch)]
model.fit(X_train, Y_train.values,
                      epochs=10, batch_size=256,
                      validation_data=(X_valid,y_valid.values),
                      verbose=2,validation_batch_size=1024,
                      callbacks=callbacks       )


#Main Train
optimizer=torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.0001,)
model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy])
scheduler =LRScheduler( torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5,min_lr=0.00001))
callbacks=[ModelCheckpoint(checkpoint,monitor='val_loss',mode='min'),scheduler]
model.fit(X_train, Y_train.values,
        epochs=30, batch_size=256,
        validation_data=(X_valid,y_valid.values),
        verbose=2,validation_batch_size=1024,
        callbacks=callbacks)


#Test It
model.load_weights(checkpoint)
logits=0
for i in range(0,64):
        logits+=pd.DataFrame(model.predict(X_test,batch_size=1024))

Test = (True_ACC(logits.idxmax(axis=1).values.reshape((-1)),y_test.values.reshape((-1))))





print('Test Score:',Test)

